# Plate solving fits file

## 모듈 설치 및 버전 확인

아래 셀을 실행하면 이 노트북을 실행하는데 필요한 모듈을 설치하고 파이썬 및 관련 모듈의 버전을 확인할 수 있습니다.

### 모듈 버전 확인

아래 셀을 실행하면 이 노트북을 실행한 파이썬 및 관련 모듈의 버전을 확인할 수 있다.

In [6]:
import importlib, sys, subprocess
packages = "ysfitsutilpy, astropy, version_information" # required modules
pkgs = packages.split(", ")

for pkg in pkgs :
    if not importlib.util.find_spec(pkg):
        # print(f"**** module {pkg} is not installed")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg, '-q'])
    else: 
        # print(f"**** module {pkg} is installed")
        pass

%load_ext version_information
import time
now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
print(f"This notebook was generated at {now} ")

vv = %version_information {packages}
for i, pkg in enumerate(vv.packages):
    print(f"{i} {pkg[0]:10s} {pkg[1]:s}")

The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information
This notebook was generated at 2024-12-09 23:39:34 (KST = GMT+0900) 
0 Python     3.12.7 64bit [GCC 11.2.0]
1 IPython    8.27.0
2 OS         Linux 6.8.0 49 generic x86_64 with glibc2.39
3 ysfitsutilpy 0.2
4 astropy    7.0.0
5 version_information 1.0.4


### import modules

In [7]:
from glob import glob
from pathlib import Path
from astropy.io import fits

import ysfitsutilpy as yfu

import _astro_utilities
import _Python_utilities

import warnings
warnings.filterwarnings('ignore')

## 환경 설정

In [8]:
#%%
#######################################################
BASEDIR = Path("/mnt/Rdata/ASTRO_data")  

PROJECDIR = BASEDIR / "C1-Variable"
TODODIR = PROJECDIR / "-_-_-_2016-_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2017-01_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2017-03_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2017-06_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2021-10_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2022-01_-_RiLA600_STX-16803_-_2bin"

PROJECDIR = BASEDIR / "C2-Asteroid"
TODODIR = PROJECDIR / "-_-_-_2022-_-_GSON300_STF-8300M_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2022-_-_RiLA600_STX-16803_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2022-_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2023-_-_GSON300_STF-8300M_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2023-_-_RiLA600_STX-16803_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2023-_-_RiLA600_STX-16803_-_2bin"

PROJECDIR = BASEDIR / "C3-EXO"
TODODIR = PROJECDIR / "-_-_-_2024-05_-_GSON300_STF-8300M_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2024-05_-_RiLA600_STX-16803_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2024-06_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2024-09_-_RiLA600_ASI6200MMPro_-_2bin"

# PROJECDIR = BASEDIR / "C4-Spectra"
# TODODIR = PROJECDIR / "-_-_-_2024-05_TEC140_ASI183MMPro_-_1bin"

DOINGDIRs = sorted(_Python_utilities.getFullnameListOfsubDirs(TODODIR))
print ("DOINGDIRs: ", format(DOINGDIRs))
print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

try : 
    BDFDIR = [x for x in DOINGDIRs if "CAL-BDF" in str(x)]
    print ("BDFDIR: ", format(BDFDIR))
    BDFDIR = Path(BDFDIR[0])    
except : 
    BDFDIR = TODODIR
    pass

DOINGDIRs = sorted([x for x in DOINGDIRs if "_LIGHT_" in str(x)])
# print ("DOINGDIRs: ", format(DOINGDIRs))
# print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

# filter_str = '2024-10-04'
# DOINGDIRs = [x for x in DOINGDIRs if filter_str in str(x)]
# remove = 'BIAS'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
# remove = 'DARK'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
# remove = 'FLAT'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
print ("DOINGDIRs: ", DOINGDIRs)
print ("len(DOINGDIRs): ", len(DOINGDIRs))
#######################################################

verbose = False

DOINGDIRs:  ['/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/-_CAL-BDF_-_2024-05_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HAT-P-23b_LIGHT_-_2024-08-03_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HAT-P-23b_LIGHT_-_2024-08-31_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HAT-P-37b_LIGHT_-_2024-06-28_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HD189733b_LIGHT_-_2024-06-16_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HD189733b_LIGHT_-_2024-06-27_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/Kepler-17b_LIGHT_-_2024-06-26_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/Qatar-10b_LIGH

In [ ]:
for DOINGDIR in DOINGDIRs[:1] :
    DOINGDIR = Path(DOINGDIR)
    if verbose == True :
        print("DOINGDIR", DOINGDIR)
    SOLVINGDIR = DOINGDIR / _astro_utilities.reduced_dir
    SOLVINGDIR = DOINGDIR / _astro_utilities.reduced_nightsky_dir
    SOLVINGDIR = DOINGDIR


    summary = yfu.make_summary(SOLVINGDIR/"*.fit*")
    if summary is not None :
        if verbose == True :
            print("len(summary):", len(summary))
            print("summary:", summary)
        #print(summary["file"][0])  
        df_light = summary.loc[summary["IMAGETYP"] == "LIGHT"].copy()
        df_light = df_light.reset_index(drop=True)
        if verbose == True :
            print("df_light:\n{}".format(df_light))
    df_light

    for _, row  in df_light.iterrows():

        fpath = Path(row["file"])
        # fpath = Path(df_light["file"][1])
        if verbose == True :
            print("fpath :" ,fpath)
        hdul = fits.open(fpath)

        if 'PIXSCALE' in hdul[0].header:
            PIXc = hdul[0].header['PIXSCALE']
        else : 
            PIXc = _astro_utilities.calPixScale(hdul[0].header['FOCALLEN'], 
                                                hdul[0].header['XPIXSZ'],
                                                hdul[0].header['XBINNING'])
        if verbose == True :
            print("PIXc : ", PIXc)
        hdul.close()

        solved = _astro_utilities.KevinSolver(fpath, 
                                                #str(SOLVEDDIR), 
                                                # downsample = 2,
                                                # pixscale = PIXc,
                                                tryASTAP = True, 
                                                tryLOCAL = False,
                                                makeLOCALsh = True,
                                                tryASTROMETRYNET = False, 
                                                verbose = False,
                                                )

All 71 keywords (guessed from /mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HAT-P-23b_LIGHT_-_2024-08-03_-_GSON300_STF-8300M_-_1bin/HAT-P-23b_LIGHT_R_2024-08-03-12-01-12_120sec_GSON300_STF-8300M_29c_1bin.fit) will be loaded.


len(summary): 122
